# Automate Excel with xlwings

## How does it work?!

![](img/xlwings_frompython_architecture.png)

In [69]:
import xlwings as xw
import numpy as np
import pandas as pd
import datetime as dt
import time
import sys

In [70]:
xw.__version__

'0.20.7'

# The Basics

## xw.view(): Excel as viewer for tabular data

In [3]:
data = np.random.rand(100, 100)
data

array([[0.42345454, 0.37221105, 0.77044255, ..., 0.31497472, 0.99651638,
        0.46932334],
       [0.99569822, 0.20574533, 0.45306278, ..., 0.03837773, 0.23818601,
        0.7761308 ],
       [0.23372558, 0.41275962, 0.22541711, ..., 0.43741382, 0.17266267,
        0.61188229],
       ...,
       [0.09611557, 0.81337375, 0.0314863 , ..., 0.00289021, 0.97928415,
        0.2107318 ],
       [0.55588759, 0.00630513, 0.47669125, ..., 0.06997784, 0.18079679,
        0.8134215 ],
       [0.72691006, 0.13002801, 0.85881172, ..., 0.29636555, 0.41852542,
        0.5037525 ]])

In [4]:
# Opens a new book
xw.view(data)

In [5]:
# Reuse an existing sheet (sheets gets cleared with every call)
xw.view(np.random.rand(5, 5), xw.sheets.active)

In [6]:
xw.view(np.random.rand(3, 3), xw.sheets.active)

## Connect to a Book

In [7]:
# Fire up a new book in the active Excel instance
wb1 = xw.Book()

In [8]:
# Connects to an unsaved book (looks in all Excel instances)
wb1 = xw.Book('Book2')

In [9]:
# Connects to a book by file name or full path and opens it if it is not open yet
# Windows: Use raw strings for path: r'C:\path\to\file.xlsx'
# Again: looks in all Excel instances

# wb1 = xw.Book('C:\\Users\\MyWorkbook.xlsx')

## Sheet object

In [10]:
sheet = wb1.sheets[0]
sheet

<Sheet [Book2]Sheet1>

## The Range object

In [11]:
# Write value
sheet.range('A1').value = 'Hello xlwings!'

In [12]:
# Read value
sheet.range('A1').value

'Hello xlwings!'

In [13]:
# Write the same value to multiple cells
sheet.range('A3:B4').value = 123

In [14]:
# Excel's numerical format is float!
sheet.range('A3').value

123.0

In [15]:
# Datetime
sheet.range('A6').value = dt.datetime(2014, 12, 9, 12, 3, 25)
sheet.range('A6').value

datetime.datetime(2014, 12, 9, 12, 3, 25)

In [16]:
# Index notation (1-based like Excel!)
sheet.range((1,1)).value

'Hello xlwings!'

In [17]:
# Formula
sheet.range('B1').formula = '=SUM(A3:B4)'

In [18]:
# Named ranges
sheet.range('B1').name = 'test'
sheet.range('test').formula

'=SUM(A3:B4)'

<div class="alert alert-info">

**Named ranges**: Named ranges give you a solid way of reading in values (e.g. parameters) that survive a reorganization of the sheet. The same is true for defining a target cell to write out values.

</div>

In [19]:
test = sheet.range('test').value
test

492.0

In [20]:
sheet.range('test').value = 'Output'

## 2d Ranges

In [21]:
sheet.range('A3:B4').value

[[123.0, 123.0], [123.0, 123.0]]

In [22]:
# Index notation
sheet.range((3,1),(4,2)).value

[[123.0, 123.0], [123.0, 123.0]]

In [23]:
# Assign a nested list to the top-left corner
sheet.range('A9').value = [['a string', 1, 2, 3],
                           [dt.datetime(2010, 1, 1), 123.5, None, None]]

In [24]:
# Range expansion: 'table', 'down', 'right'
# Correspond to Ctrl-Shift-down and/or right
# They return a Range object!
sheet.range('A9').expand('table')

<Range [Book2]Sheet1!$A$9:$D$10>

In [25]:
# 'table' is default
sheet.range('A9').expand().value

[['a string', 1.0, 2.0, 3.0],
 [datetime.datetime(2010, 1, 1, 0, 0), 123.5, None, None]]

In [26]:
# Use .clear() to also clear the formatting
sheet.range('A9').expand().clear_contents()

## 1d vectors

In [27]:
# Horizontal...
sheet.range('A12').value = [1, 2, 3, 4]

In [28]:
# ... and vertical vectors
sheet.range('A13').options(transpose=True).value = [5, 6, 7, 8]
# this is the same as:
# sheet.range('A13').value = [[5], [6], [7], [8]]

In [29]:
sheet.range('A12').expand('right').value

[1.0, 2.0, 3.0, 4.0]

In [30]:
sheet.range('A12').expand('down').value

[1.0, 5.0, 6.0, 7.0, 8.0]

## ndim

In [31]:
sheet.range('A12').options(ndim=2, expand='right').value

[[1.0, 2.0, 3.0, 4.0]]

In [32]:
sheet.range('A12').options(ndim=2, expand='down').value

[[1.0], [5.0], [6.0], [7.0], [8.0]]

## Autofit

In [33]:
# autofit columns and rows based on single Cell
sheet.range('A3').autofit()

In [34]:
# autofit columns based on Range
sheet.range('A1:C3').columns.autofit()

In [35]:
# autofit a whole column
sheet.range('A:A').autofit()

## Background color

In [36]:
# Assign an RGB tuple
sheet.range('A1').color = (0, 255, 0)
sheet.range('A1').color

(0, 255, 0)

# Range indexing/slicing

In [37]:
rng = sheet.range('A1:D5')
rng[0, 0]

<Range [Book2]Sheet1!$A$1>

In [38]:
rng[1]

<Range [Book2]Sheet1!$B$1>

In [39]:
rng[:, 3:]

<Range [Book2]Sheet1!$D$1:$D$5>

In [40]:
rng[1:3, 1:3]

<Range [Book2]Sheet1!$B$2:$C$3>

In [41]:
xw.books.active.close()

# Full qualification

In [72]:
# Get all availabe PIDs (Process Ids)
xw.apps.keys()

[14821]

In [74]:
xw.apps[pid].books

Books([<Book [Ações_Escolha-Online.xlsx]>, <Book [output20201017.xlsx]>])

In [43]:
# This allows us to specificy a specific Excel instance
pid = xw.apps.keys()[0] # or you could use xw.apps.active.pid

In [44]:
# We start at the app (=Excel instance) and walk our way down to the range
xw.apps[pid].books[0].sheets[0].range('A1')

<Range [Ações_Escolha-Online.xlsx]Planilha1!$A$1>

In [45]:
# Square brackets behave like in Python, whereas round brackets behave like in Excel:
xw.apps(pid).books(1).sheets(1).range('A1')

<Range [Ações_Escolha-Online.xlsx]Planilha1!$A$1>

In [46]:
# Instead of indices we can also use names:
xw.apps[pid].books['Book1'].sheets['Sheet1'].range('A1')
xw.apps(pid).books('Book1').sheets('Sheet1').range('A1')

<Range [Book1]Sheet1!$A$1>

## Work with multiple apps

In [47]:
app1 = xw.apps[pid]
app2 = xw.App()

In [48]:
# Open the same workbook twice in different Excel instances
app1.books.open('timeseries.xlsx')
app2.books.open('timeseries.xlsx')

<Book [timeseries.xlsx]>

In [49]:
# xw.Book('timeseries.xlsx')  # this will throw an error

In [50]:
# The following syntax is *required* if the same file is open in >1 instances (full qualification)
print(app1.books['timeseries.xlsx'])
print(app2.books['timeseries.xlsx'])
print(app1.books['timeseries.xlsx'].app)
print(app2.books['timeseries.xlsx'].app)

<Book [timeseries.xlsx]>
<Book [timeseries.xlsx]>
<Excel App 14821>
<Excel App 29747>


# Active Objects

In [51]:
# Active app
xw.apps.active

<Excel App 29747>

In [52]:
# active book in active app
xw.books.active

<Book [timeseries.xlsx]>

In [53]:
# active sheet in active book in active app
xw.sheets.active

<Sheet [timeseries.xlsx]Sheet1>

In [54]:
# This is a special shortcut for interactive use only:
# It takes the active sheet from the active book
xw.Range('A1').value

'Date'

In [55]:
app2.kill()

<div class="alert alert-info">

**Note**: Active objects are meant to be used in interactive use. Don't use them in scripts as this would not be reliable. This is especially true for `xw.Range`. For scripts, always go through a sheet object: `sheet.range(...)`.


</div>

# Sheets

In [56]:
xw.sheets[0].name

'Sheet1'

In [57]:
xw.sheets.count  # or: len(xw.sheets)

1

In [58]:
xw.sheets.add(name='New', after='Sheet1')

<Sheet [timeseries.xlsx]New>

### Range shortcuts: sheet[...]

In [59]:
sheet = xw.sheets[0]
sheet['A1']  # same as: sheet.range('A1')

<Range [timeseries.xlsx]Sheet1!$A$1>

In [60]:
sheet['A1:B5']

<Range [timeseries.xlsx]Sheet1!$A$1:$B$5>

In [61]:
sheet[0, 1]

<Range [timeseries.xlsx]Sheet1!$B$1>

In [62]:
sheet[:10, :10]

<Range [timeseries.xlsx]Sheet1!$A$1:$J$10>

# Excel Charts

In [63]:
wb = xw.Book()
sheet = wb.sheets[0]
sheet.range('A1').value = [['one', 'two'],
                           [1.1, 2.2],
                           [3.3, None]]

In [64]:
chart = sheet.charts.add()
chart.set_source_data(sheet.range('A1').expand())
chart.chart_type = 'line'
chart.top = sheet.range('A5').top

In [65]:
chart.chart_type = 'area'

In [66]:
# available chart types (we're only printing the first 10 here)
xw.constants.chart_types[:10]

('3d_area',
 '3d_area_stacked',
 '3d_area_stacked_100',
 '3d_bar_clustered',
 '3d_bar_stacked',
 '3d_bar_stacked_100',
 '3d_column',
 '3d_column_clustered',
 '3d_column_stacked',
 '3d_column_stacked_100')

In [67]:
wb.close()

# Matplotlib

In [68]:
%matplotlib inline
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import matplotlib

# Swap rate example
years = [1, 2, 3, 4, 5, 7, 10]
swap_rate = [0.0079, 0.0094, 0.0107, 0.0119,
             0.013, 0.0151, 0.0174]
years_new = np.linspace(1, 10, num=10)
interpolate = interp1d(years, swap_rate, kind='quadratic')

fig = plt.figure(figsize=(6, 4))
swaprate_plot = plt.plot(years, swap_rate, 'o',
                         years_new, interpolate(years_new), '-')

ModuleNotFoundError: No module named 'scipy'

In [ ]:
wb = xw.Book()
sheet = wb.sheets[0]

In [ ]:
plot = sheet.pictures.add(fig, name='SwapRate', update=True)

In [ ]:
# Fine Tuning
width, height = fig.get_size_inches()
dpi = fig.get_dpi()
sheet.pictures.add(fig, name='SwapRate2', update=True,
                   left=sheet.range('A25').left, top=sheet.range('A25').top,
                   width=width * dpi / 2, height=height * dpi / 2)

In [ ]:
# Alternatively manipulate properties after adding the picture
plot.height = plot.height / 2
plot.width = plot.width / 2

In [ ]:
wb.close()

<div class="alert alert-info">

**Note**: If you set `update=True`, you can resize and position the plot on Excel: subsequent calls to `pictures.add()` with the same name  will update the picture without changing its position or size.


</div>

# Table objects

Excel Table objects aren't officially supported yet, but reading actually works nicely:

In [ ]:
# The sample book has a table that was created with:
# Insert > Table (incl. Header Row and Total Row)
wb = xw.Book('table_objects.xlsx')
sheet = wb.sheets[0]

In [ ]:
# Get entire table body - no different from named range
sheet.range('Table1').value

In [ ]:
# Get column data 
sheet.range('Table1[Symbol]').value

In [ ]:
# Get column including header and total rows
sheet.range('Table1[[#All], [Last]]').value

In [ ]:
# Get header row for one column
sheet.range('Table1[[#Headers], [Last]]').value

In [ ]:
# Total row for one column
sheet.range('Table1[[#Totals], [Last]]').value

In [ ]:
# Two or more adjancent columns
sheet.range('Table1[[Index]:[Last]]').value

In [ ]:
wb.close()

# Efficiency

<div class="alert alert-info">

**Watch out**: Minimize your cross-application calls to improve efficiency, i.e. if possible, always read and write 2d ranges instead of single cells: `sht.range('A1').value = [[1,2],[3,4]]` instead of `sheet.range('A1').value = 1`, `sheet.range('B1').value = 2` etc. 

</div>

In [ ]:
wb = xw.Book()
sheet = wb.sheets[0]

In [ ]:
# This calls each cell individually - Don't do this!
for i, cell in enumerate(sheet.range('A1:E30')):
    cell.value = i

In [ ]:
sheet.range('A1').expand().clear()

In [ ]:
# Do this instead
import numpy as np
sheet.range('A1').value = np.arange(5 * 30).reshape((30, 5))

In [ ]:
wb.close()


# Workaround for missing features: Manipulate the underlying object

In [ ]:
wb = xw.Book()
sheet = wb.sheets[0]

In [ ]:
# On Windows, the underlying object is a pywin32 COM object
# On Mac, the underlying object is an appscript object

sheet.range('A1').api  # same for the other objects

## E.g. Range.ClearFormats() is not implemented yet

In [ ]:
sheet.range('A10').value = 1
sheet.range('A10').color = (255, 0, 0)

In [ ]:
# Code that makes use of the api property will be platform dependent (!)
if sys.platform.startswith('darwin'):
    # Mac version (appscript syntax)
    sheet.range('A10').api.clear_formats()
elif sys.platform.startswith('win'):
    # Windows version (pywin32 syntax)
    sheet.range('A10').api.ClearFormats()

# Calling VBA macros

Paste the following code into a VBA module:
```
Function MySum(x, y)
    MySum = x + y
End Function
```
Then run you can call it from Python:

In [ ]:
wb = xw.books.active
my_sum = wb.macro('MySum')
my_sum(1, 2)